In [1]:
import pandas as pd

dfalles = pd.read_csv('merged_df_influentdata.csv')

dflabo = pd.read_excel('LaboPython.xlsx', sheet_name='Blad2')
dflabo['ratioCOD'] = dflabo['CODtprimeff'] / dflabo['CODsprimeff']
dflabo['ratioN'] = dflabo['TNprimeff'] / dflabo['NH4primeff']
mean_ratioCOD = dflabo['ratioCOD'].mean()
mean_ratioN = dflabo['ratioN'].mean()

print(mean_ratioCOD)
print(mean_ratioN)

dfCODt = dfalles[['Date', 'CODtprimeff']]
dfCODs = dfalles[['Date', 'CODsprimeff']]
dfTSS = dfalles[['Date', 'TSSprimeff']]
dfQwaste = dfalles[['Date', 'Qwaste']]

2.2982105479133383
1.3833857499407898


In [2]:
dfCODt = dfCODt.copy()
dfCODs = dfCODs.copy()

dfCODt['Date'] = pd.to_datetime(dfCODt['Date'])
dfCODs['Date'] = pd.to_datetime(dfCODs['Date'])

In [3]:
mask = (dfCODt['Date'] >= "2024-04-20") & (dfCODt['Date'] < "2024-07-31")
dfCODt.loc[mask, 'CODtprimeff'] = dfCODs.loc[mask, 'CODsprimeff'] * mean_ratioCOD

print(dfCODs.loc[mask, :].shape)
print(dfCODt.loc[mask, :].shape)

print(dfCODt)

(102, 2)
(102, 2)
          Date  CODtprimeff
0   2023-10-25   232.000000
1   2023-10-26   231.142857
2   2023-10-27   230.285714
3   2023-10-28   229.428571
4   2023-10-29   228.571429
..         ...          ...
519 2025-03-27   221.000000
520 2025-03-28   221.000000
521 2025-03-29   221.000000
522 2025-03-30   221.000000
523 2025-03-31   221.000000

[524 rows x 2 columns]


In [4]:
import pandas as pd

# Werk met een kopie om veilig te zijn
dfCODt_clean = dfCODt.copy()

# Zet 'Date' naar 12:00
dfCODt_clean['Date'] = pd.to_datetime(dfCODt_clean['Date']).dt.normalize() + pd.Timedelta(hours=12)

# Zet index
dfCODt_clean = dfCODt_clean.set_index("Date")

# Maak tijdgrid vanaf 25 okt 00:00 tot einde + 1 dag
start = pd.Timestamp("2023-10-25 00:00")
end = dfCODt_clean.index.max() + pd.Timedelta(days=1)
full_range = pd.date_range(start=start, end=end, freq="10min")
full_range = full_range[full_range < end]

# Reindex + interpolatie (extrapolatie toegestaan)
dfCODt_interp = dfCODt_clean.reindex(full_range)
dfCODt_interp = dfCODt_interp.interpolate(method="time", limit_direction='both')

# Reset index
dfCODt_interp = dfCODt_interp.reset_index().rename(columns={"index": "Datetime"})

print(dfCODt_interp)

dfCODt_interp.to_csv("CODt_interpolated.csv", index=False)

                 Datetime  CODtprimeff
0     2023-10-25 00:00:00        232.0
1     2023-10-25 00:10:00        232.0
2     2023-10-25 00:20:00        232.0
3     2023-10-25 00:30:00        232.0
4     2023-10-25 00:40:00        232.0
...                   ...          ...
75523 2025-04-01 11:10:00        221.0
75524 2025-04-01 11:20:00        221.0
75525 2025-04-01 11:30:00        221.0
75526 2025-04-01 11:40:00        221.0
75527 2025-04-01 11:50:00        221.0

[75528 rows x 2 columns]


In [5]:
import pandas as pd

# Werk met een kopie om veilig te zijn
dfTSS_clean = dfTSS.copy()

# Zet 'Date' naar 12:00
dfTSS_clean['Date'] = pd.to_datetime(dfTSS_clean['Date']).dt.normalize() + pd.Timedelta(hours=12)

# Zet index
dfTSS_clean = dfTSS_clean.set_index("Date")

# Maak tijdgrid vanaf 25 okt 00:00 tot max datum + 1 dag
start = pd.Timestamp("2023-10-25 00:00")
end = dfTSS_clean.index.max() + pd.Timedelta(days=1)
full_range = pd.date_range(start=start, end=end, freq="10min")
full_range = full_range[full_range < end]

# Reindex + interpolatie (extrapolatie toegestaan)
dfTSS_interp = dfTSS_clean.reindex(full_range)
dfTSS_interp = dfTSS_interp.interpolate(method="time", limit_direction='both')

# Reset index
dfTSS_interp = dfTSS_interp.reset_index().rename(columns={"index": "Datetime"})

# Opslaan als CSV
dfTSS_interp.to_csv("TSS_interpolated.csv", index=False)

# (Optioneel) Print controle
print(dfTSS_interp.head())


             Datetime  TSSprimeff
0 2023-10-25 00:00:00   41.699752
1 2023-10-25 00:10:00   41.699752
2 2023-10-25 00:20:00   41.699752
3 2023-10-25 00:30:00   41.699752
4 2023-10-25 00:40:00   41.699752


In [6]:
import pandas as pd

# Zorg dat Date een datetime is
dfQWaste = dfQwaste.copy()
dfQWaste['Date'] = pd.to_datetime(dfQWaste['Date'])
dfQWaste['Date'] = dfQWaste['Date'].dt.floor('D')  # naar middernacht

# Bepaal tijdsbereik
start = dfQWaste['Date'].min()
end = dfQWaste['Date'].max() + pd.Timedelta(days=1)

# Maak tijdgrid
full_range = pd.date_range(start=start, end=end, freq='10min')
full_range = full_range[full_range < end]  # verwijder eventueel eindtijd

# Zet index
dfQWaste = dfQWaste.set_index('Date')

# Reindex en forward fill
dfQWaste_ffill = dfQWaste.reindex(full_range).ffill()

# Reset index en hernoem
dfQWaste_ffill = dfQWaste_ffill.reset_index().rename(columns={'index': 'Datetime'})

# Opslaan
dfQWaste_ffill.to_csv("QWaste_ffill.csv", index=False)

# Bekijk resultaat
print(dfQWaste_ffill.head())

             Datetime  Qwaste
0 2023-10-25 00:00:00   280.0
1 2023-10-25 00:10:00   280.0
2 2023-10-25 00:20:00   280.0
3 2023-10-25 00:30:00   280.0
4 2023-10-25 00:40:00   280.0
